<a href="https://colab.research.google.com/github/rajagopal17/GenAI/blob/main/Agents_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#In this notebook we will explore agents in llamaindex with openai embeddings and meta/llama3 model as llm

link to github repo for llamaindex

https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/crewai_llamaindex.ipynb




In [1]:

########################
!pip install llama_index

#######################
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-openai
import openai
from google.colab import userdata
import os
import json
userdata.get('LM_API_KEY')

userdata.get('LlamaCloud')

userdata.get('OPEN_API_KEY')
userdata.get('TVLY_API_KEY')
os.environ["OPEN_API_KEY"] = userdata.get('OPEN_API_KEY')
openai.api_key = os.environ["OPEN_API_KEY"]

####################################
import textwrap
import llama_index
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,SimpleKeywordTableIndex
from llama_index.core import ServiceContext, StorageContext,load_index_from_storage
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core import download_loader
from llama_index.core.response.notebook_utils import display_response
#####################################


#Store VectorStoreIndex to local disk for later use
from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex
from llama_index.core.indices import SummaryIndex
from llama_index.core import StorageContext

#import llama retrievers

from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import get_response_synthesizer
#####################################
from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
#embed_model = OpenAIEmbedding(embed_batch_size=10)
Settings.embed_model = embed_model

# get API key and create embeddings
from llama_index.embeddings.openai import OpenAIEmbedding


#llm         = OpenAI(model="gpt-3.5-turbo")

###### Import loggine ###############
import logging
import sys


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:

###############load documents
documents = SimpleDirectoryReader("/content/drive/MyDrive/Assignment9").load_data()

#Settings.chunk_size = 1024
#nodes = Settings.node_parser.get_nodes_from_documents(documents)


################ initialize storage context (by default it's in-memory)
#storage_context = StorageContext.from_defaults()
#storage_context.docstore.add_documents(nodes)

############### Create a  vector store index.  VectorStoreIndex only needs an embed model
#index = VectorStoreIndex.from_documents(documents, embed_model=Settings.embed_model)
#############Store vectorindex
from llama_index.core.storage.storage_context import StorageContext
PERSIST_DIR     = "/content/drive/MyDrive/llama_storage/metallama"
#index.storage_context.persist(persist_dir=PERSIST_DIR)
storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)

###############reload index from storage
sap_index       = load_index_from_storage(storage_context=storage_context)

#Create a summary index from the same set of documents loaded
summary_index = SummaryIndex.from_documents(documents)

In [ ]:

#meta-llama/Llama-3-8b-chat-hf
!pip install together
!pip install llama-index-llms-together
#!pip install llama-index-embeddings-together
import together
together.api_key = userdata.get('TG_API_KEY')


from llama_index.llms.together import TogetherLLM
Settings.llm    = TogetherLLM(model="meta-llama/Llama-3-8b-chat-hf", api_key=together.api_key)
llm             = Settings.llm
#llm_ai         = OpenAI(model="gpt-3.5-turbo")

#Queries using LLAMA 3 8b

#Create vector_tool and summary_tool

In [4]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
                  query_engine =summary_index.as_query_engine(response_mode="tree_summarize",use_async=True),
                  description  =("Useful for summarizing the requested details from the context ")
                                             )

vector_tool  = QueryEngineTool.from_defaults(
                  query_engine  =sap_index.as_query_engine(),
                  description   =("Useful for retrieving specific context in detail  ")
                                             )




In [5]:
question = '''provide a summary overview for the field depreciation key
              in asset master in SAP and then give details for each
              field in depreciation key in bullet points.
              Please answer based on the provided context only
              and do not guess'''

In [8]:
from IPython.display import Markdown, display
# vector_tool   -> this tool can be used for querying and produces better results based on prompt
# summary_tool  -> this tool will not use llm, outputs similiar documents and need to collect them into a list for downstream applications

vec_query=vector_tool.query_engine.query(question).response  #

print(vec_query)

Summary Overview:
The Depreciation Key in the Asset Master in SAP is a component that enables the automatic calculation of depreciation types, such as ordinary depreciation, special depreciation, and unplanned depreciation. It controls the calculation of asset values and interest calculation for imputed interest in cost accounting.

Details for each field in the Depreciation Key:

• Depreciation Key: This field specifies the depreciation key that is used for the automatic calculation of depreciation.
• Depreciation Method: This field specifies the method used for calculating depreciation, such as linear or declining balance.
• Depreciation Period: This field specifies the period over which the depreciation is calculated, such as monthly or annually.
• Depreciation Rate: This field specifies the rate at which the depreciation is calculated, such as a percentage or a fixed amount.
• Interest Calculation Method: This field specifies the method used for calculating interest, such as simple

#Router query engine

In [34]:
#RouterQueryEngine provides both summary and details from the request - did not find it much useful

from llama_index.core.query_engine import RouterQueryEngine

single_query = RouterQueryEngine.from_defaults([vector_tool, summary_tool],select_multi=False)
multi_query  = RouterQueryEngine.from_defaults([vector_tool, summary_tool],select_multi=True)


#single_response = single_query.query("what is depreciation key in SAP  from the given context")
multi_response  = multi_query.query(question)

#print('Single_response: ')
#display_response(single_response)

print('muliple_response')
print(multi_response)
#print(f'Summary:\n------------------> \n\n {single_response} \n\n Detailed overview:\n----------------->\n\n {multi_response}')

muliple_response
Summary Overview:
The Depreciation Key in the Asset Master in SAP is a component that enables the automatic calculation of depreciation types, such as ordinary depreciation, special depreciation, and unplanned depreciation. It controls the calculation of asset values and interest calculation for cost accounting.

Details for each field in the Depreciation Key:

• Depreciation Key: This field specifies the depreciation key that is used for the automatic calculation of depreciation.
• Depreciation Method: This field defines the method used for calculating depreciation, such as linear or declining balance.
• Depreciation Period: This field specifies the period over which the depreciation is calculated, such as monthly or annually.
• Depreciation Rate: This field defines the rate at which the depreciation is calculated, such as a percentage or a fixed amount.
• Interest Calculation: This field specifies whether interest is calculated for the asset, and if so, the method us

In [41]:

def vector_query(query: str) -> str:

    """Perform a vector search over an index and provide detailed overview.
    query (str): the string query to be embedded.
    """
    result = vector_tool.query_engine.query(query).response
    return result


def multiple_query(query: str) -> str:

    """Perform a vector search over an index and provide detailed overview.
    query (str): the string query to be embedded.
    """
    result1 = multi_query.query(query).response
    return (result1)


In [43]:
user_query =''' what is asset class and explain
               its importance in detail in asset accounting
               with an example '''

print(display(Markdown(vector_query(user_query))))

An asset class is a means of structuring fixed assets in the Asset Accounting component. It is a way to categorize assets according to the requirements of an enterprise, and it applies to all company codes in a client. An asset class can provide default values to all asset master records in the class, making it possible to create new asset master records simply and without errors.

The importance of an asset class lies in its ability to establish a connection between the asset master records and the corresponding accounts in the general ledger in Financial Accounting. This connection is created by the account determination key in the asset class. The asset class also determines the screen layout, tab layout, and field characteristics of the asset master record, as well as the assignment of asset numbers.

For example, let's say a company has different types of assets, such as buildings, machinery, and vehicles, which require different valuation parameters. In this case, the company can define multiple asset classes, each with its own set of default values and account determination keys. This allows the company to structure its assets according to its specific requirements and to track the depreciation of each asset class separately.

In addition, the asset class is a selection criterion in all standard reports in Asset Accounting, and it can also be used for sorting and totaling by class-specific characteristics. This makes it a useful tool for analyzing and reporting on the company's fixed assets.

Overall, the asset class is a critical component of Asset Accounting, as it provides a way to structure and categorize fixed assets according to the company's specific requirements, and it enables the company to track and report on its assets in a more detailed and accurate manner.

None


In [46]:
print(multiple_query('''what is period control in asset master
                        in SAP  from the given context. Provide
                        answer in bullet points'''))



Here is the answer to the query:

• For determining the depreciation start and end dates for asset transactions, you can set an appropriate period control in the period control method.
• This enables you to set the depreciation start date for all acquisitions within the same year to the beginning of the year, for example.
• You can also set the depreciation start date for retirements to the first or last day of each period.
• The system determines the start date or end date of depreciation calculation using the period control and the asset value date of a transaction (acquisition or retirement).


#CrewAI

In [ ]:
# Set up OpenAI
!pip install -U langchain-community tavily-python
#import openai
openai.api_key = userdata.get('OPEN_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TVLY_API_KEY')
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()


In [ ]:
news_now = tool.invoke({"query": "What happened in India 2024 andhra pradesh elections?"})

In [ ]:
news_now[3]['content']

'Andhra Pradesh (AP) Election Results 2024 Live Updates: The counting of votes will be kicking off at 8 am) Andhra Pradesh Lok Sabha and Assembly Election Results 2024: Celebrations broke out at Telugu Desam Party (TDP) offices across Andhra Pradesh as the party after in a reversal of the 2019 mandate, the ruling YSRCP government, led by Chief Minister YS Jagan Mohan Reddy, was decimated by the ...'